En el proceso de limpieza y preparación del dataset se eliminaron 1.454 filas que no tenían descripción de producto y 4.432 filas con cantidades o precios negativos, ya que representaban registros inválidos que afectarían el análisis. Se completaron los valores nulos en CustomerID asignando identificadores únicos, y se aseguró que InvoiceDate estuviera correctamente formateada como fecha. Además, se añadieron varias columnas derivadas para enriquecer el análisis: TotalPrice (resultado de Quantity × UnitPrice), AvgQuantityByProduct (promedio de unidades vendidas por código de producto), TotalQuantityByCountry (ventas agregadas por país), MonetaryDemand (demanda monetaria estimada), y variables temporales como InvoiceMonth e InvoiceDay. También se aplicó One-Hot Encoding para transformar la columna Country en variables binarias, facilitando su uso en modelos de machine learning y aprendizaje reforzado. Finalmente, el dataset quedó sin nulos, sin valores negativos, con una estructura enriquecida para análisis avanzados y con versiones guardadas tanto en formato limpio como codificado.

In [70]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [71]:
import pandas as pd

# Intentar con codificación y lectura optimizada
file_path = r"c:\Users\anaca\OneDrive\Escritorio\hackaton\01_data\input\Online Retail.csv"

# Leer solo primeras filas para inspección inicial
df_sample = pd.read_csv(file_path, encoding='ISO-8859-1', nrows=1000)
print("=== Primeras filas ===")
print(df_sample.head())

# Si quieres, leer todo el archivo con menos memoria
df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

# Información general
print("\n=== Info ===")
print(df.info())

# Estadísticas numéricas
print("\n=== Estadísticas ===")
print(df.describe())

# Valores nulos
print("\n=== Valores nulos ===")
print(df.isnull().sum())


=== Primeras filas ===
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  

=== Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       -----

In [72]:
df = df.dropna(subset=['Description'])


existing_ids = set(df['CustomerID'].dropna().astype(int).unique())
num_missing = df['CustomerID'].isnull().sum()
max_id = int(df['CustomerID'].max())
new_ids = np.random.choice(range(max_id+1, max_id+1+num_missing*10), size=num_missing, replace=False)
new_ids = new_ids[:num_missing]
df.loc[df['CustomerID'].isnull(), 'CustomerID'] = new_ids
df['CustomerID'] = df['CustomerID'].astype(int)

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True, errors='coerce')

df = df.dropna(subset=['InvoiceDate'])

negatives = df[(df['Quantity'] < 0) | (df['UnitPrice'] < 0)]
print(f"Registros con Quantity o UnitPrice negativos: {len(negatives)}")

print("\nValores nulos por columna después de limpieza:")
print(df.isnull().sum())
print("\nPrimeras filas del dataset limpio:")
print(df.head())

Registros con Quantity o UnitPrice negativos: 4432

Valores nulos por columna después de limpieza:
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Primeras filas del dataset limpio:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-01-12 08:26:00       2.55       17850  United Kingdom  
1 2010-01-12 08:26:00       3.39       17850  United Kingdom  
2 2010-01-12 08:26:00       2.75       17850  United Kingdom  
3 2010-01-12 08:26:00       3.3

In [73]:
# Convertir Quantity y UnitPrice negativos a 0
df['Quantity'] = df['Quantity'].apply(lambda x: x if x > 0 else 0)
df['UnitPrice'] = df['UnitPrice'].apply(lambda x: x if x > 0 else 0)

# Verificar
print("Registros con Quantity o UnitPrice negativos después de ajuste:", 
      len(df[(df['Quantity'] < 0) | (df['UnitPrice'] < 0)]))


Registros con Quantity o UnitPrice negativos después de ajuste: 0


In [74]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Función para detectar outliers usando IQR
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    print(f"{column} → Outliers: {len(outliers)}")
    return outliers

# Detectar outliers en Quantity, UnitPrice y TotalPrice
outliers_quantity = detect_outliers_iqr(df, 'Quantity')
outliers_unitprice = detect_outliers_iqr(df, 'UnitPrice')
outliers_totalprice = detect_outliers_iqr(df, 'TotalPrice')

Quantity → Outliers: 23851
UnitPrice → Outliers: 17074
TotalPrice → Outliers: 18219
UnitPrice → Outliers: 17074
TotalPrice → Outliers: 18219


In [75]:
def cap_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    data[column] = data[column].apply(lambda x: upper if x > upper else (lower if x < lower else x))

cap_outliers(df, 'Quantity')
cap_outliers(df, 'UnitPrice')
cap_outliers(df, 'TotalPrice')


In [76]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Función para detectar outliers usando IQR
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    print(f"{column} → Outliers: {len(outliers)}")
    return outliers

# Detectar outliers en Quantity, UnitPrice y TotalPrice
outliers_quantity = detect_outliers_iqr(df, 'Quantity')
outliers_unitprice = detect_outliers_iqr(df, 'UnitPrice')
outliers_totalprice = detect_outliers_iqr(df, 'TotalPrice')

Quantity → Outliers: 0
UnitPrice → Outliers: 0
TotalPrice → Outliers: 13285


In [77]:
print("=== Estadísticas descriptivas numéricas ===")
print(df.describe())

# Información general del dataframe
print("\n=== Información general del dataframe ===")
print(df.info())

# Columnas categóricas
categorical_cols = df.select_dtypes(include=['object']).columns
print("\n=== Columnas categóricas y número de valores únicos ===")
for col in categorical_cols:
    print(f"{col} → {df[col].nunique()} valores únicos")

# Ejemplo de los valores más frecuentes por columna categórica
print("\n=== Valores más frecuentes por columna categórica ===")
for col in categorical_cols:
    print(f"\nColumna: {col}")
    print(df[col].value_counts().head(10))

=== Estadísticas descriptivas numéricas ===
            Quantity                 InvoiceDate      UnitPrice    CustomerID  \
count  232301.000000                      232301  232301.000000  2.323010e+05   
mean        6.523625  2011-05-14 05:52:51.527200       2.964615  1.879302e+05   
min         0.000000         2010-01-12 08:26:00       0.000000  1.234700e+04   
25%         1.000000         2011-03-04 11:36:00       1.250000  1.439500e+04   
50%         3.000000         2011-06-09 11:58:00       2.100000  1.634700e+04   
75%        10.000000         2011-09-06 11:35:00       4.130000  5.268800e+04   
max        23.500000         2011-12-10 17:19:00       8.450000  1.354439e+06   
std         7.113935                         NaN       2.397713  3.528726e+05   

          TotalPrice  
count  232301.000000  
mean       13.040838  
min         0.000000  
25%         3.360000  
50%         8.450000  
75%        16.800000  
max       198.575000  
std        16.567512  

=== Información ge

In [78]:
latin_america_countries = [
    'Argentina', 'Brasil', 'Chile', 'Colombia', 'México', 'Perú', 
    'Uruguay', 'Venezuela', 'Ecuador', 'Bolivia', 'Paraguay', 'Costa Rica'
]


In [79]:
import numpy as np

# Número de registros
n = len(df)

# Generar un array repetido de países que cubra todo el dataset
repeated_countries = np.tile(latin_america_countries, n // len(latin_america_countries) + 1)

# Mezclar aleatoriamente para no tener bloques
np.random.seed(42)  # Para reproducibilidad
np.random.shuffle(repeated_countries)

# Asignar solo hasta el tamaño del dataset
df['Country'] = repeated_countries[:n]

# Verificar distribución
print(df['Country'].value_counts())


Country
Bolivia       19359
Paraguay      19359
Brasil        19359
Perú          19359
Uruguay       19359
Venezuela     19359
Colombia      19359
Ecuador       19359
Argentina     19358
Chile         19357
Costa Rica    19357
México        19357
Name: count, dtype: int64


In [80]:
df.head()   
df.describe()
df = df.dropna(subset=['Description'])


existing_ids = set(df['CustomerID'].dropna().astype(int).unique())
num_missing = df['CustomerID'].isnull().sum()
max_id = int(df['CustomerID'].max())
new_ids = np.random.choice(range(max_id+1, max_id+1+num_missing*10), size=num_missing, replace=False)
new_ids = new_ids[:num_missing]
df.loc[df['CustomerID'].isnull(), 'CustomerID'] = new_ids
df['CustomerID'] = df['CustomerID'].astype(int)

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True, errors='coerce')

df = df.dropna(subset=['InvoiceDate'])

negatives = df[(df['Quantity'] < 0) | (df['UnitPrice'] < 0)]
print(f"Registros con Quantity o UnitPrice negativos: {len(negatives)}")

print("\nValores nulos por columna después de limpieza:")
print(df.isnull().sum())
print("\nPrimeras filas del dataset limpio:")
print(df.head())

df.info()

Registros con Quantity o UnitPrice negativos: 0

Valores nulos por columna después de limpieza:
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
TotalPrice     0
dtype: int64

Primeras filas del dataset limpio:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER       6.0   
1    536365     71053                  WHITE METAL LANTERN       6.0   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER       8.0   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE       6.0   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.       6.0   

          InvoiceDate  UnitPrice  CustomerID   Country  TotalPrice  
0 2010-01-12 08:26:00       2.55       17850   Bolivia       15.30  
1 2010-01-12 08:26:00       3.39       17850   Bolivia       20.34  
2 2010-01-12 08:26:00       2.75       17850     Chile       2

In [81]:

exchange_rates = {
    'Argentina': 350,    # Pesos argentinos
    'Brasil': 5.3,       # Reales
    'Chile': 820,        # Pesos chilenos
    'Colombia': 5000,    # Pesos colombianos
    'México': 18.5,      # Pesos mexicanos
    'Perú': 3.8,         # Soles
    'Uruguay': 38,       # Pesos uruguayos
    'Venezuela': 24_000, # Bolívares
    'Ecuador': 1,        # Dólar estadounidense
    'Bolivia': 6.9,      # Bolivianos
    'Paraguay': 7.2,     # Guaraníes
    'Costa Rica': 540    # Colones
}


In [82]:
def convert_currency(row):
    rate = exchange_rates.get(row['Country'], 1)  # 1 si no se encuentra
    row['UnitPrice'] = row['UnitPrice'] * rate
    row['TotalPrice'] = row['TotalPrice'] * rate
    return row


df = df.apply(convert_currency, axis=1)


In [83]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-01-12 08:26:00,17.595,17850,Bolivia,105.570
1,536365,71053,WHITE METAL LANTERN,6.0,2010-01-12 08:26:00,23.391,17850,Bolivia,140.346
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-01-12 08:26:00,2255.000,17850,Chile,18040.000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-01-12 08:26:00,24.408,17850,Paraguay,146.448
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-01-12 08:26:00,17.967,17850,Brasil,107.802


In [84]:
# Demanda promedio por producto
df['AvgQuantityByProduct'] = df.groupby('StockCode')['Quantity'].transform('mean')

# Demanda acumulada por país
df['TotalQuantityByCountry'] = df.groupby('Country')['Quantity'].transform('sum')


In [85]:
# Demanda monetaria: cuánto gastan los clientes por producto
df['MonetaryDemand'] = df['Quantity'] * df['UnitPrice']  # ya en moneda local


In [86]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

df['InvoiceHour'] = df['InvoiceDate'].dt.hour
df['InvoiceMinute'] = df['InvoiceDate'].dt.minute
df['InvoiceSecond'] = df['InvoiceDate'].dt.second

# Combinación útil: hora:minuto
df['InvoiceHourMinute'] = df['InvoiceDate'].dt.strftime("%H:%M")


In [87]:
print("Valores únicos de InvoiceHour:")
print(sorted(df['InvoiceHour'].unique()))

print("\nValores únicos de InvoiceMinute:")
print(sorted(df['InvoiceMinute'].unique()))

print("\nValores únicos de InvoiceSecond:")
print(sorted(df['InvoiceSecond'].unique()))

print("\nPrimeros 30 valores únicos de InvoiceHourMinute:")
print(sorted(df['InvoiceHourMinute'].unique())[:30])


Valores únicos de InvoiceHour:
[np.int32(7), np.int32(8), np.int32(9), np.int32(10), np.int32(11), np.int32(12), np.int32(13), np.int32(14), np.int32(15), np.int32(16), np.int32(17), np.int32(18), np.int32(19), np.int32(20)]

Valores únicos de InvoiceMinute:
[np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5), np.int32(6), np.int32(7), np.int32(8), np.int32(9), np.int32(10), np.int32(11), np.int32(12), np.int32(13), np.int32(14), np.int32(15), np.int32(16), np.int32(17), np.int32(18), np.int32(19), np.int32(20), np.int32(21), np.int32(22), np.int32(23), np.int32(24), np.int32(25), np.int32(26), np.int32(27), np.int32(28), np.int32(29), np.int32(30), np.int32(31), np.int32(32), np.int32(33), np.int32(34), np.int32(35), np.int32(36), np.int32(37), np.int32(38), np.int32(39), np.int32(40), np.int32(41), np.int32(42), np.int32(43), np.int32(44), np.int32(45), np.int32(46), np.int32(47), np.int32(48), np.int32(49), np.int32(50), np.int32(51), np.int32(52), np.int32

In [88]:
df['AvgUnitPriceByProduct'] = df.groupby('StockCode')['UnitPrice'].transform('mean')
df['PriceDiff'] = df['UnitPrice'] - df['AvgUnitPriceByProduct']


In [89]:
# Número total de compras por cliente
df['CustomerTotalPurchases'] = df.groupby('CustomerID')['InvoiceNo'].transform('nunique')

# Gasto total por cliente (moneda local)
df['CustomerTotalSpend'] = df.groupby('CustomerID')['TotalPrice'].transform('sum')


In [90]:
df['DemandElasticity'] = df['Quantity'] / df['UnitPrice']  # simple proxy


In [91]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,AvgQuantityByProduct,...,MonetaryDemand,InvoiceHour,InvoiceMinute,InvoiceSecond,InvoiceHourMinute,AvgUnitPriceByProduct,PriceDiff,CustomerTotalPurchases,CustomerTotalSpend,DemandElasticity
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-01-12 08:26:00,17.595,17850,Bolivia,105.570,8.810413,...,105.570,8,26,0,08:26,8106.808683,-8089.213683,35,1.045909e+07,0.341006
1,536365,71053,WHITE METAL LANTERN,6.0,2010-01-12 08:26:00,23.391,17850,Bolivia,140.346,4.842105,...,140.346,8,26,0,08:26,14391.582871,-14368.191871,35,1.045909e+07,0.256509
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-01-12 08:26:00,2255.000,17850,Chile,18040.000,5.713740,...,18040.000,8,26,0,08:26,11072.155130,-8817.155130,35,1.045909e+07,0.003548
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-01-12 08:26:00,24.408,17850,Paraguay,146.448,6.489083,...,146.448,8,26,0,08:26,11165.095406,-11140.687406,35,1.045909e+07,0.245821
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-01-12 08:26:00,17.967,17850,Brasil,107.802,6.203629,...,107.802,8,26,0,08:26,15102.017694,-15084.050694,35,1.045909e+07,0.333946


In [92]:
import pandas as pd

# Aplicar One-Hot Encoding a la columna 'Country'
df_encoded = pd.get_dummies(df, columns=['Country'], prefix='Country')

# Convertir todas las columnas generadas de booleano a entero (0 y 1)
df_encoded = df_encoded.astype({col: 'int' for col in df_encoded.columns if 'Country_' in col})

# Verificar las primeras filas
print(df_encoded.head())
df_encoded.info()
df_encoded.describe()

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER       6.0   
1    536365     71053                  WHITE METAL LANTERN       6.0   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER       8.0   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE       6.0   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.       6.0   

          InvoiceDate  UnitPrice  CustomerID  TotalPrice  \
0 2010-01-12 08:26:00     17.595       17850     105.570   
1 2010-01-12 08:26:00     23.391       17850     140.346   
2 2010-01-12 08:26:00   2255.000       17850   18040.000   
3 2010-01-12 08:26:00     24.408       17850     146.448   
4 2010-01-12 08:26:00     17.967       17850     107.802   

   AvgQuantityByProduct  TotalQuantityByCountry  ...  Country_Brasil  \
0              8.810413                126514.5  ...               0   
1              4.842105                126514.5  ...          

c:\Users\anaca\OneDrive\Escritorio\hackaton\venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Quantity,InvoiceDate,UnitPrice,CustomerID,TotalPrice,AvgQuantityByProduct,TotalQuantityByCountry,MonetaryDemand,InvoiceHour,InvoiceMinute,...,Country_Brasil,Country_Chile,Country_Colombia,Country_Costa Rica,Country_Ecuador,Country_México,Country_Paraguay,Country_Perú,Country_Uruguay,Country_Venezuela
count,232301.000000,232301,232301.000000,2.323010e+05,2.323010e+05,232301.000000,232301.000000,2.323010e+05,232301.000000,232301.000000,...,232301.000000,232301.000000,232301.000000,232301.000000,232301.000000,232301.000000,232301.000000,232301.000000,232301.000000,232301.000000
mean,6.523625,2011-05-14 05:52:51.527200,7605.873753,1.879302e+05,3.359537e+04,6.523625,126287.053215,3.359537e+04,13.075996,29.952471,...,0.083336,0.083327,0.083336,0.083327,0.083336,0.083327,0.083336,0.083336,0.083336,0.083336
min,0.000000,2010-01-12 08:26:00,0.000000,1.234700e+04,0.000000e+00,0.000000,124909.500000,0.000000e+00,7.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,2011-03-04 11:36:00,11.385000,1.439500e+04,3.784800e+01,4.090659,125604.500000,3.784800e+01,11.000000,16.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,2011-06-09 11:58:00,60.840000,1.634700e+04,2.453400e+02,5.990909,126314.000000,2.453400e+02,13.000000,30.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,10.000000,2011-09-06 11:35:00,1732.500000,5.268800e+04,6.519000e+03,8.066372,127032.500000,6.519000e+03,15.000000,44.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,23.500000,2011-12-10 17:19:00,202800.000000,1.354439e+06,4.765800e+06,23.500000,127937.500000,4.765800e+06,20.000000,59.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,7.113935,NaN,25879.872396,3.528726e+05,1.452398e+05,3.498198,810.095697,1.452398e+05,2.501488,16.850383,...,0.276390,0.276377,0.276390,0.276377,0.276390,0.276377,0.276390,0.276390,0.276390,0.276390


In [93]:
import os
import pandas as pd


output_dir = r'c:\Users\anaca\OneDrive\Escritorio\hackaton\01_data\processed'
os.makedirs(output_dir, exist_ok=True)


df.to_csv(os.path.join(output_dir, 'OnlineRetail_clean.csv'), index=False)
print("Versión limpia guardada correctamente.")


df_encoded = pd.get_dummies(df, columns=['Country'], prefix='Country')
df_encoded = df_encoded.astype({col: 'int' for col in df_encoded.columns if 'Country_' in col})

df_encoded.to_csv(os.path.join(output_dir, 'OnlineRetail_encoded.csv'), index=False)
print("Versión con One-Hot Encoding guardada correctamente.")


Versión limpia guardada correctamente.
Versión con One-Hot Encoding guardada correctamente.
Versión con One-Hot Encoding guardada correctamente.
